In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
train_data_file = "./zhengqi_train.txt"
test_data_file = "./zhengqi_test.txt"

In [ ]:
train_data = pd.read_csv(train_data_file, sep="\t", encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')
train_data.info() # 数据的行列信息

In [ ]:
train_data.describe(); # 数据的统计信息：均值、方差、4分位值等

In [ ]:
fig = plt.figure(figsize=(4,6))
sns.boxplot(train_data['V0'], orient="v", width=0.5) #绘制箱线图

# column = train_data.columns.tolist()[:39]
# fig = plt.figure(figsize=(80,60), dpi=75)
# for i in range(38):
#     plt.subplot(7,8,i+1)
#     sns.boxplot(train_data[column[i]], orient='v', width=0.5)
#     plt.ylabel(column[i], fontsize=36)

In [ ]:
def find_outliers(model, X, y, sigma=3):
    """
    找出异常值
    """
    try:
        # 生成预测值
        y_pred = pd.Series(model.predict(X), index=y.index)
    except:
        model.fit(X, y)
        y_pred = pd.Series(model.predict(X), index=y.index)
        
    # 误差    
    resid = y - y_pred
    # 误差均值
    mean_resid = resid.mean()
    # 误差的标准差
    std_resid = resid.std()
    
    z = (resid - mean_resid)/std_resid
    
    #找出边界点
    outliers = z[abs(z)>sigma].index
    
    print('R2=',model.score(X, y))
    
    # 均方差
    print("mse", mean_squared_error(y,y_pred))
    print('--------------------------------------------')
    
    print('mean of residuals:', mean_resid)
    print('std of residuals:', std_resid)
    print('---------------------------------------------')
    
    print(len(outliers),'outliers:')
    print(outliers.tolist())
    
    #y,y_pred
    plt.figure(figsize=(15,5))
    ax_131 = plt.subplot(1,3,1)
    plt.plot(y,y_pred, '.') #异常
    plt.plot(y.loc[outliers], y_pred.loc[outliers], 'ro') #不接受
    plt.legend(['Accepted','Outlier'])
    plt.xlabel('y')
    plt.ylabel('y_pred')
    
    #y,y-pred
    ax_132 = plt.subplot(1,3,2)
    plt.plot(y,y-y_pred, '.') #异常
    plt.plot(y.loc[outliers], y.loc[outliers]-y_pred.loc[outliers],'ro')  #不接受
    plt.legend(['Accepted','Outlier'])
    plt.xlabel('y')
    plt.ylabel('y - y_read')
    
    #值
    ax_133 = plt.subplot(1,3,3)
    z.plot.hist(bins=50, ax=ax_133) #异常
    z.loc[outliers].plot.hist(color='r', bins=50, ax=ax_133)  #不接受
    plt.legend(['Accepted', 'Outlier'])
    plt.xlabel('z')
    
    plt.savefig('outliers.png')
    
    return outliers

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
X_train = train_data.iloc[:,0:-1]
y_train = train_data.iloc[:,-1]
# STEP1 采用**模型预测的模式**找出异常值 
# -> 其思想是，采用一个标准模型对训练集数据进行预测，然后排除
# 掉偏离训练数据（y）过多的数据
outliers = find_outliers(Ridge(), X_train, y_train)

In [ ]:
#STEP2 统计假设校验

# 统计方法一般都有其适用的条件，或者说是必须满足的。
# 使用线性回归需要满足线性、独立性、正态性、方差齐性、自变量间不存在多重共线、因变量为连续变量。
# 不考虑前提条件地生搬硬套，也不对模型进行诊断，只能是“Garbage in，garbage out”。

[线性回归中的正态分布](https://mp.weixin.qq.com/s?__biz=MzIzNjk2NDg4NA==&mid=2247484629&idx=1&sn=b542cc03895de8aea58f63f574159f60&chksm=e8ce99aedfb910b8861cdb0b161c0c18d12a196921e8c5fd808334afdef6fa16756d80649aa2&token=2074505415&lang=zh_CN#rd)


In [ ]:
# 假设1: 正态性校验

# 直方图和Q-Q图
# 指数据的分位数和正态分布分为数对比参照的图，如果数据符合正态分布，则所有的点都会落在直线上
# 查看V0是否近似正态分布
plt.figure(figsize=(10,5))

ax = plt.subplot(1,2,1)
sns.distplot(train_data['V0'], fit=stats.norm)
ax=plt.subplot(1,2,2)
res = stats.probplot(train_data['V0'], plot=plt)
train_cols = 6
train_rows = len(train_data.columns)
plt.figure(figsize=(4*train_cols, 4*train_rows))

i = 1

for col in train_data.columns:
    i+=1
    ax = plt.subplot(train_rows, train_cols, i)
    sns.distplot(train_data[col],fit=stats.norm)
    
    i+=1
    
    ax=plt.subplot(train_rows, train_cols, i)
    res = stats.probplot(train_data[col], plot=plt)
    
plt.tight_layout()
plt.show()

# ...数据分布不是正态的，数据并不跟随对角线分布，后续可以使用数据变换对这些数据进行处理。

In [ ]:
# 假设2: 训练数据和测试数据同分布校验
# KDE分布图 核密度估计
# 可以理解为是对直方图对加窗平滑。通过绘制KDE分布图，可以查看并对比训练集和测试集中特征变量对分布情况，
# 发现两个数据集中分布不一致的特征变量

# 首先对比同一特征变量V0在训练集和测试集中的分布情况，并查看数据分布是否一致。

plt.figure(figsize=(8,4), dpi=150)
ax = sns.kdeplot(train_data['V0'], color="Red", shade=True)
ax = sns.kdeplot(test_data['V0'], color="Blue",shade=True)
ax.set_xlabel('V0')
ax.set_ylabel('Frequency')
ax = ax.legend(["train","test"])

# 可以看到V0在两个数据集中的分布基本一致。

In [ ]:
# 然后对比所有变量在训练集和测试集中的KDE分布

dist_cols = 6
dist_rows = len(test_data.columns)
plt.figure(figsize=(4 * dist_cols, 4 * dist_rows))
i = 1
for col in test_data.columns:
    ax = plt.subplot(dist_rows, dist_cols, i)
    ax = sns.kdeplot(train_data[col], color="Red", shade=True)
    ax = sns.kdeplot(test_data[col], color="Blue", shade=True)
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")
    ax = ax.legend(["train","test"])
    i += 1
plt.show()    

# 可以看到变量A。。。在训练集和测试集中的分布不一致，这会导致模型的**泛化能力**变差，需要删除此类特征。

In [ ]:
#假设3: 分析变量之间的线性回归关系
# 线性回归关系图
# 分析变量之间的线性回归关系。


# 首先查看*特征变量*V0与*target*变量的线性回归关系



fcols = 2
frows = 1

plt.figure(figsize=(8,4), dpi=150)

ax = plt.subplot(1,2,1)
sns.regplot(x='V0', y='target', data=train_data, ax=ax,
           scatter_kws={'marker':'.','s':3,'alpha':0.3}, line_kws={'color':'k'})
plt.xlabel('V0')
plt.ylabel('target')

ax = plt.subplot(1,2,2)
sns.distplot(train_data['V0'].dropna())
plt.xlabel('V0')

plt.show()






In [ ]:
fcols = 6
frows = len(test_data.columns)
plt.figure(figsize=(5*fcols, 4*frows))

i = 0
for col in test_data.columns:
    i += 1
    ax = plt.subplot(frows, fcols, i)
    sns.regplot(x = col, y='target', data=train_data, ax=ax,
               scatter_kws={'marker':'.', 's':3, 'alpha':0.3}, line_kws={'color':'k'});
    plt.xlabel(col)
    plt.ylabel('target')
    
    i+=1
    
    ax = plt.subplot(frows, fcols, i)
    sns.distplot(train_data[col].dropna())
    plt.xlabel(col)
    
plt.show()    

[线性回归中的正态分布](https://mp.weixin.qq.com/s?__biz=MzIzNjk2NDg4NA==&mid=2247484629&idx=1&sn=b542cc03895de8aea58f63f574159f60&chksm=e8ce99aedfb910b8861cdb0b161c0c18d12a196921e8c5fd808334afdef6fa16756d80649aa2&token=2074505415&lang=zh_CN#rd)

In [ ]:

pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows',10)

# 删除训练集和测试集中**分布不一致**的特征变量
data_train1 = train_data.drop([
    'V5','V9','V11','V17','V22','V28'
], axis=1)

train_corr = data_train1.corr()

# 计算**剩余**特征变量及target变量的相关性系数

ax = plt.subplots(figsize=(20,16))
ax = sns.heatmap(train_corr, vmax=.8, square=True, annot=True)
annot = True

In [ ]:
# 根据相关系数筛选特征变量

k = 10 # number of variables for heatmap
cols = train_corr.nlargest(k, 'target')['target'].index

cm = np.corrcoef(train_data[cols].values.T)
hm = plt.subplots(figsize=(10,10))
hm = sns.heatmap(train_data[cols].corr(), annot=True, square=True)
plt.show()

In [ ]:
# 然后找出与target变量的相关系数大于0.5的特征变量

threshold = 0.5

corrmat = train_data.corr()
top_corr_features = corrmat.index[abs(corrmat["target"])>threshold]
plt.figure(figsize=(10,10))
g = sns.heatmap(train_data[top_corr_features].corr(), annot=True, cmap='RdYlGn')

In [ ]:
threshold = 0.5

corr_matrix = data_train1.corr().abs() # 取相关性绝对值
drop_col = corr_matrix[corr_matrix["target"]<threshold].index
# data_all.drop(drop_col, axis=1, inplace=True) # 删除不重要的特征

In [ ]:
# Box-Cox变换
# 由于线性回归是基于正态分布的，因此在进行统计分析时，需要将数据转换使其符合正态分布


# 在连续的响应变量不满足正态分布时，可以使用Box-Cox变换，这一变换可以使线性回归模型在满足线性、正态性、独立性及方差齐性
# 的同时又不丢失信息


drop_columns = ['V5','V9','V11','V17','V22','V28']

train_x = train_data.drop(['target'], axis=1)

data_all = pd.concat([train_x, test_data])

data_all.drop(drop_columns, axis=1, inplace=True)
data_all.head()

In [ ]:
# 对合并后的每列数据进行归一化：

cols_numeric = list(data_all.columns)

def scale_minmax(col):
    return (col - col.min())/(col.max() - col.min())

data_all[cols_numeric] = data_all[cols_numeric].apply(scale_minmax, axis=0)
data_all[cols_numeric].describe()

In [ ]:
train_data_process = train_data[cols_numeric]
train_data_process =  train_data_process[cols_numeric].apply(scale_minmax,axis=0)

test_data_process = train_data[cols_numeric]
test_data_process = test_data_process[cols_numeric].apply(scale_minmax, axis=0)

In [ ]:
cols_numeric_left = cols_numeric[0:13]
cols_numeric_right = cols_numeric[13:]
train_data_process = pd.concat([train_data_process, train_data['target']], axis=1)


fcols = 6
frows = len(cols_numeric_left)
plt.figure(figsize=(4 * fcols, 4*frows))

i = 0

for var in cols_numeric_left:
    dat = train_data_process[[var,'target']].dropna()
    i += 1
    plt.subplot(frows, fcols, i)
    sns.distplot(dat[var], fit=stats.norm)
    plt.title(var + ' Original')
    plt.xlabel('')
    
    i+=1
    plt.subplot(frows, fcols, i)
    _ =  stats.probplot(dat[var], plot=plt)
    plt.title('skew='+'{:.4f}'.format(stats.skew(dat[var])))
    plt.xlabel('')
    plt.ylabel('')
    
    i+=1
    plt.subplot(frows, fcols,i)
    plt.plot(dat[var], dat['target'], '.', alpha=0.5)
    try:
        plt.title('corr='+ '{:.2f}'.format(np.corrcoef(dat[var],dat['target'])[0][1]))
    except Exception  as e:
        print("error {}".format(e))
    
    i+=1
    plt.subplot(frows, fcols, i)
    #boxcox 变换
    trans_var, lambda_var = stats.boxcox(dat[var].dropna()+1)
    trans_var = scale_minmax(trans_var)
    sns.distplot(trans_var, fit=stats.norm)
    plt.title(var + ' Transformed')
    plt.xlabel('')
    
    i+=1
    plt.subplot(frows, fcols, i)
    _ = stats.probplot(trans_var, plot=plt)
    plt.title('skew='+'{:.4f}'.format(stats.skew(trans_var)))
    plt.xlabel('')
    plt.ylabel('')
    
    i+=1
    plt.subplot(frows, fcols, i)
    plt.plot(trans_var, dat['target'], '.', alpha=0.5)
    try:
        plt.title('corr='+ '{:.2f}'.format(np.corrcoef(trans_var,dat['target'])[0][1]))
    except Exception as e:
        print("error {}".format(e))

    
    
